In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split


input_file = "/Users/evgeny/Desktop/study/Text2EmojiRus/data/final/final_train.csv"
df = pd.read_csv(input_file)

texts = df['text'].values
emojis_data = df['emoji'].factorize()
emojis = emojis_data[0]
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, emojis, test_size=0.1, random_state=42)

/var/folders/dt/y7wk4rzx3dsg1r6_m1tgxhwc0000gn/T/ipykernel_7446/3742441719.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
import torch
import torch.nn as nn
from gensim.models import Word2Vec
import spacy

# Загрузка модели spaCy для русского языка
nlp = spacy.load("ru_core_news_sm")

def preprocess_text(text):
    doc = nlp(text.lower())
    return [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

def tokenize_data(texts):
    return [preprocess_text(text) for text in texts]

val_tokenized = tokenize_data(val_texts)
print("val processed")
train_tokenized = tokenize_data(train_texts)



val processed


In [6]:
import numpy as np 
from tqdm import tqdm

model = Word2Vec(sentences=train_tokenized, vector_size=100, window=5, min_count=1, workers=4)

def texts_to_vectors(texts, model):
    vectors = []
    for text in tqdm(texts, desc="Converting texts to vectors", unit="text"):
        if len(text) > 0:
            # Создаем список векторов слов
            word_vectors = [model.wv[word] for word in text if word in model.wv]
            if word_vectors:  # Проверяем, что список не пустой
                # Вычисляем среднее значение векторов слов
                vector = np.mean(word_vectors, axis=0)
            else:
                # Если нет известных слов, используем вектор нулей
                vector = np.zeros(model.vector_size)
        else:
            # Если текст пустой, используем вектор нулей
            vector = np.zeros(model.vector_size)
        vectors.append(vector)
    
    # Преобразуем список numpy arrays в единый numpy array, затем в тензор PyTorch
    return torch.tensor(np.array(vectors), dtype=torch.float32)



train_vectors = texts_to_vectors(train_tokenized, model)
val_vectors = texts_to_vectors(val_tokenized, model)

train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

class EmojiDataset(torch.utils.data.Dataset):
    def __init__(self, vectors, labels):
        self.vectors = vectors
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {'vector': self.vectors[idx], 'labels': self.labels[idx]}

train_dataset = EmojiDataset(train_vectors, train_labels)
val_dataset = EmojiDataset(val_vectors, val_labels)

Converting texts to vectors: 100%|██████████| 44656/44656 [00:00<00:00, 74734.69text/s]


In [14]:
train_dataset[0]['vector'].shape

torch.Size([100])

In [7]:
class EmojiClassifier(nn.Module):
    def __init__(self, input_size, num_labels):
        super(EmojiClassifier, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_labels)
        )

    def forward(self, vector, labels=None):
        logits = self.classifier(vector)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
            return loss, logits
        return logits



In [15]:
from torch.utils.data import DataLoader
from transformers import AdamW


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = EmojiClassifier(input_size=100, num_labels=len(df['emoji'].unique())).to(device)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

optimizer = AdamW(model.parameters(), lr=2e-4)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [17]:
from tqdm import tqdm
import torch
from tqdm import tqdm
import numpy as np

num_classes = 62


def train_model(model, train_loader, val_loader, epochs=10):

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        loop = tqdm(train_loader, leave=True)

        for batch in loop:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs[0]
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loop.set_description(f'Epoch [{epoch+1}/{epochs}]')
            loop.set_postfix(loss=loss.item())

        # Логируем тренировочную потерю в TensorBoard
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch + 1}, Loss: {avg_loss}')

        model.eval()
        correct = 0
        total = 0
        correct_top10 = 0
        class_total = np.zeros(num_classes)
        class_correct = np.zeros(num_classes)
        class_predicted = np.zeros(num_classes) # num of elements that were predicted to this class


        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validating"):
                batch = {k: v.to(device) for k, v in batch.items()}
                logits = model(**batch)[1]
                predictions = torch.argmax(logits, dim=-1)
                correct += (predictions == batch['labels']).sum().item()
                total += len(batch['labels'])

                top10_preds = torch.topk(logits, k=10, dim=-1).indices
                correct_top10 += sum([batch['labels'][i] in top10_preds[i] for i in range(len(batch['labels']))])

                for i in range(len(batch['labels'])):
                    label = batch['labels'][i].item()
                    pred = predictions[i].item()
                    class_predicted[pred] += 1
                    class_total[label] += 1
                    if label == pred:
                        class_correct[label] += 1


        top1_acc = correct / total * 100
        top10_acc = correct_top10 / total * 100

        print(f'Validation Accuracy (Top-1): {top1_acc:.2f}%')
        print(f'Validation Accuracy (Top-10): {top10_acc:.2f}%')

        for i in range(num_classes):
            if class_total[i] > 0 and class_predicted[i] > 0:
                recall = 100 * class_correct[i] / class_total[i]
                prec = 100 * class_correct[i] / class_predicted[i]
                print(f'Class {i} Recall: {recall:.2f}% ({int(class_correct[i])}/{int(class_total[i])})      Precision: {prec:.2f}% ({int(class_correct[i])}/{int(class_predicted[i])})') # num of elements from this class that were predicted correctly
            else:
                print(f'Class {i} Accuracy: N/A (нет примеров для этого класса) class_predicted {class_predicted[i]}, class_total {class_total[i]}, class_correct {class_correct[i]}')

In [18]:
train_model(model, train_loader, val_loader)

Epoch [1/10]: 100%|██████████| 1570/1570 [00:09<00:00, 169.86it/s, loss=3.26]


Epoch 1, Loss: 3.4584106424052243


Validating: 100%|██████████| 698/698 [00:00<00:00, 1270.51it/s]


Validation Accuracy (Top-1): 16.34%
Validation Accuracy (Top-10): 59.55%
Class 0 Recall: 37.20% (748/2011)      Precision: 18.55% (748/4033)
Class 1 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 389.0, class_correct 0.0
Class 2 Recall: 1.07% (12/1118)      Precision: 14.81% (12/81)
Class 3 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 351.0, class_correct 0.0
Class 4 Recall: 2.87% (44/1531)      Precision: 10.11% (44/435)
Class 5 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 338.0, class_correct 0.0
Class 6 Recall: 83.39% (4256/5104)      Precision: 16.97% (4256/25077)
Class 7 Recall: 15.22% (563/3699)      Precision: 11.71% (563/4806)
Class 8 Recall: 0.00% (0/678)      Precision: 0.00% (0/6)
Class 9 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 215.0, class_correct 0.0
Class 10 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total

Epoch [2/10]: 100%|██████████| 1570/1570 [00:07<00:00, 196.67it/s, loss=3.35]


...

Validation Accuracy (Top-1): 16.84%
Validation Accuracy (Top-10): 61.33%
Class 0 Recall: 41.12% (827/2011)      Precision: 18.09% (827/4571)
Class 1 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 389.0, class_correct 0.0
Class 2 Recall: 3.31% (37/1118)      Precision: 18.69% (37/198)
Class 3 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 351.0, class_correct 0.0
Class 4 Recall: 2.87% (44/1531)      Precision: 10.26% (44/429)
Class 5 Recall: 0.00% (0/338)      Precision: 0.00% (0/2)
Class 6 Recall: 81.74% (4172/5104)      Precision: 17.42% (4172/23954)
Class 7 Recall: 13.54% (501/3699)      Precision: 12.20% (501/4107)
Class 8 Recall: 0.74% (5/678)      Precision: 20.83% (5/24)
Class 9 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 215.0, class_correct 0.0
Class 10 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 387.0, class_correct 0.0
Class 11 Recall: 0.11% (2

Epoch [10/10]: 100%|██████████| 1570/1570 [00:07<00:00, 207.81it/s, loss=3.12]


Epoch 10, Loss: 3.341740140034135


Validating: 100%|██████████| 698/698 [00:00<00:00, 1210.48it/s]

Validation Accuracy (Top-1): 16.97%
Validation Accuracy (Top-10): 61.17%
Class 0 Recall: 43.01% (865/2011)      Precision: 18.76% (865/4610)
Class 1 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 389.0, class_correct 0.0
Class 2 Recall: 4.11% (46/1118)      Precision: 16.08% (46/286)
Class 3 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 351.0, class_correct 0.0
Class 4 Recall: 4.90% (75/1531)      Precision: 11.90% (75/630)
Class 5 Recall: 0.00% (0/338)      Precision: 0.00% (0/2)
Class 6 Recall: 81.96% (4183/5104)      Precision: 17.39% (4183/24051)
Class 7 Recall: 16.65% (616/3699)      Precision: 11.67% (616/5280)
Class 8 Recall: 0.74% (5/678)      Precision: 21.74% (5/23)
Class 9 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 215.0, class_correct 0.0
Class 10 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 387.0, class_correct 0.0
Class 11 Recall: 0.16% (3

In [21]:
import numpy as np
import torch
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Предполагаем, что функции предобработки текста и создания векторов уже определены
# ...
model = Word2Vec(sentences=train_tokenized, vector_size=100, window=5, min_count=1, workers=4)

def texts_to_vectors(texts, model):
    vectors = []
    for text in tqdm(texts, desc="Converting texts to vectors", unit="text"):
        if len(text) > 0:
            word_vectors = [model.wv[word] for word in text if word in model.wv]
            if word_vectors:
                vector = np.mean(word_vectors, axis=0)
            else:
                vector = np.zeros(model.vector_size)
        else:
            vector = np.zeros(model.vector_size)
        vectors.append(vector)
    return np.array(vectors)

# Предполагаем, что у нас есть train_texts, train_labels, val_texts, val_labels
# ...

# Создаем векторные представления текстов
print("Converting training texts to vectors...")
train_vectors = texts_to_vectors(train_texts, model)
print("Converting validation texts to vectors...")
val_vectors = texts_to_vectors(val_texts, model)

train_labels = train_labels.numpy() if isinstance(train_labels, torch.Tensor) else np.array(train_labels)
val_labels = val_labels.numpy() if isinstance(val_labels, torch.Tensor) else np.array(val_labels)



# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    train_vectors, train_labels, test_size=0.2, random_state=42
)

# Инициализируем и обучаем модель CatBoost
print("Training CatBoost model...")
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.001,
    depth=6,
    loss_function='MultiClass',
    eval_metric='Accuracy',
    random_seed=42,
    verbose=100
)

model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=20)

# Делаем предсказания на валидационной выборке
print("Making predictions on validation set...")
val_predictions = model.predict(val_vectors)

# Оцениваем производительность модели
accuracy = accuracy_score(val_labels, val_predictions)
print(f"Validation accuracy: {accuracy:.4f}")

print("Classification Report:")
print(classification_report(val_labels, val_predictions))

Converting training texts to vectors...


Converting texts to vectors: 100%|██████████| 401903/401903 [00:07<00:00, 53654.05text/s]


Converting validation texts to vectors...


Converting texts to vectors: 100%|██████████| 44656/44656 [00:00<00:00, 54369.02text/s]


Training CatBoost model...
0:	learn: 0.1144307	test: 0.1141563	best: 0.1141563 (0)	total: 3.92s	remaining: 1h 5m 21s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.1141563305
bestIteration = 0

Shrink model to first 1 iterations.
Making predictions on validation set...
Validation accuracy: 0.1143
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2011
           1       0.00      0.00      0.00       389
           2       0.00      0.00      0.00      1118
           3       0.00      0.00      0.00       351
           4       0.00      0.00      0.00      1531
           5       0.00      0.00      0.00       338
           6       0.11      1.00      0.21      5104
           7       0.11      0.00      0.00      3699
           8       0.00      0.00      0.00       678
           9       0.00      0.00      0.00       215
          10       0.00      0.00      0.00       387
         

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  